In [2]:
#-------------------------------------------------------------------------------------------------------------------------------
#
#                                                    CAMERON MUTCH
#                                              CLEANING TEST SUBMISSION
#
#-------------------------------------------------------------------------------------------------------------------------------

import pandas as pd
import numpy as np
import sqlite3
from sqlite3 import OperationalError

#                                              imports all dependencies
#-------------------------------------------------------------------------------------------------------------------------------

def remove_blank_answers(dataframe):
    dataframe = dataframe.fillna ({
        'I watch tv': 'No',
        'I use social media': 'No',
        'I use streaming services': 'No',
        'I read magazines': 'No',
        'I read newspapers': 'No',
        'gender': 'Male',
        'age': dataframe['age'].mean(skipna = True).round(0)
    })
    
    return dataframe 

#                        function which fills all blank cells in the database
#                        with estimations of what they should be:
#                        answers left blank are assumed to be a no
#                        blank gender is assumed to be male (in raw data 500 female and 494 male, 500 500 split assumed)
#                        blank age is filled with the average age rounded to the nearest whole number

#                        this method ensures no data is lost, but can introduce small bias. If it were important that
#                        absolutely no bias was introduced, then rows with blank cells could just be removed with a line like
#                        dataframe = dataframe.dropna()
#-------------------------------------------------------------------------------------------------------------------------------

def same_market_format(dataframe):
    dataframe['market'] = dataframe.apply(
        lambda row: 'USA' if row['id'] <= 500 else row['market'],
        axis=1
    )    
    dataframe['market'] = dataframe.apply(
        lambda row: 'UK' if row['id'] >= 501 else row['market'],
        axis=1
    ) 
    
    return dataframe

#                        function which makes sure all of the responses in the market column are formatted exactly the
#                        same way such that there are only UK and USA as options, eliminating things like 'United States'
#-------------------------------------------------------------------------------------------------------------------------------

def complete_data(dataframe):
    dataframe['number of different media used']=np.NaN
    dataframe['number of different media used']=dataframe['number of different media used'].fillna(0)

    dataframe['number of different media used'] = dataframe.apply(
        lambda row: row['number of different media used']+1 if row['I watch tv'] == 'Yes' else row['number of different media used'],
        axis=1
    ) 
    dataframe['number of different media used'] = dataframe.apply(
        lambda row: row['number of different media used']+1 if row['I use social media'] == 'Yes' else row['number of different media used'],
        axis=1
    ) 
    dataframe['number of different media used'] = dataframe.apply(
        lambda row: row['number of different media used']+1 if row['I use streaming services'] == 'Yes' else row['number of different media used'],
        axis=1
    ) 
    dataframe['number of different media used'] = dataframe.apply(
        lambda row: row['number of different media used']+1 if row['I read magazines'] == 'Yes' else row['number of different media used'],
        axis=1
    ) 
    dataframe['number of different media used'] = dataframe.apply(
        lambda row: row['number of different media used']+1 if row['I read newspapers'] == 'Yes' else row['number of different media used'],
        axis=1
    ) 
    
    return dataframe

#                        function which creates a new column, calls it 'number of different media used', fills it with
#                        NaN data, replaces the NaN data with 0's and then uses if statements to count how many forms of media
#                        are being used and totals them in the new column
#-------------------------------------------------------------------------------------------------------------------------------    

def create_sql_database():    
    conn = sqlite3.connect('table_for_cleaned_data.db')
    c = conn.cursor()
    
    fd = open('data_table.SQL', 'r')
    sqlFile = fd.read()
    fd.close()
    
    sqlCommands = sqlFile.split(';')

    try:
        c.execute(sqlCommands[0])
    except:
        print('command skipped')
    
    c.close()
    conn.close()
    
#                        function which creates a local sql database and calls it 'table_for_cleaned_data',
#                        it then opens the sql file 'data_table.SQL' and executes the code stored in there.
#                        any errors are handled by the except statement (comments for SQL code in 'data_table.SQL')
#-------------------------------------------------------------------------------------------------------------------------------
    
def run_data_cleaning_script(file_name: str):
    
    df = pd.read_csv(r'raw_data.csv')        
    

    df_without_nulls = remove_blank_answers(df)
    
    df_without_inconsistency = same_market_format(df_without_nulls)
    
    df_complete = complete_data(df_without_inconsistency)

    df_complete.to_csv('cleaned_data.csv',index = False)
    
    create_sql_database()

#                        main function which calls all the functions and then writes the resulting database into a new
#                        csv file called 'cleaned_data.csv'
#-------------------------------------------------------------------------------------------------------------------------------    
run_data_cleaning_script(r'raw_data.csv')

#                        this line was included for testing purposes and can be ignored
#-------------------------------------------------------------------------------------------------------------------------------